In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as sps

In [ ]:
Higgs_Mass = 125.1  # In Units of MeV / c**2
Higgs_Width = 4.2   # In Units of MeV / c**2

mass_grid = np.linspace(100., 160., 61)

In [ ]:
from functools import partial

def Gauss(x, nsig, mu, sigma):
    return nsig*sps.norm(loc=mu, scale=sigma).pdf(x)

def poly1(x, ref_mass, offset, slope):
    return offset + (x-ref_mass)*slope

def model_func(x, ref_mass, nsig, mu, sigma, offset, slope):
    return Gauss(x, nsig, mu, sigma) + poly1(x, ref_mass, offset, slope)

def generic_chi2(params, data_vals, model, x, ref_mass):
    model_vals = model(x, ref_mass, *params)
    return np.sum((data_vals - model_vals)**2)

def cost_func(data_vals, model, x, ref_mass):
    return partial(generic_chi2, data_vals=data_vals, model=model, x=x, ref_mass=ref_mass)

In [ ]:
def passed_cuts(cut_width, mass_grid, model_sig, model_bkg):
    mask = np.abs(mass_grid - 125) < cut_width
    n_sig = np.sum(model_sig[mask])
    n_bkg = np.sum(model_bkg[mask])
    return n_sig, n_bkg

In [ ]:
def plot_nexp(mass_grid, model_sig, model_bkg):
    sig_cts = np.zeros(26)
    bkg_cts = np.zeros(26)
    widths = np.linspace(0, 25, 26)
    for i, width in enumerate(widths):
        sig_cts[i], bkg_cts[i] = passed_cuts(width, mass_grid, model_sig, model_bkg)
    _ = plt.plot(widths, sig_cts)
    _ = plt.plot(widths, bkg_cts)
    _ = plt.yscale('log')

In [ ]:
def find_sig2noise(mass_grid, model_sig, model_bkg, plot=True):
    sig_cts = np.zeros(26)
    bkg_cts = np.zeros(26)
    widths = np.linspace(0, 25, 26)
    for i, width in enumerate(widths):
        if i == 0:
            continue
        sig_cts[i], bkg_cts[i] = passed_cuts(width, mass_grid, model_sig, model_bkg)
    sig2noise = np.zeros(26)
    sig2noise[1:] = sig_cts[1:]/np.sqrt(bkg_cts[1:])
    if plot:
        _ = plt.plot(widths, sig_cts/np.sqrt(bkg_cts))
    return sig2noise

In [ ]:
ref_mass = 130.
nsig_per_month = 30.
nbkg_per_mev_per_month = 40.
bkg_slope_per_mev_per_month = -0.2

In [ ]:
model_bkg = poly1(mass_grid, ref_mass, nbkg_per_mev_per_month, bkg_slope_per_mev_per_month)
model_sig = Gauss(mass_grid, nsig_per_month, Higgs_Mass, Higgs_Width)

In [ ]:
_ = plt.plot(mass_grid, model_sig)
_ = plt.plot(mass_grid, model_bkg)